AEDM - 7476

In [1]:
# Needs Uni enviornment

In [2]:
import requests
from bs4 import BeautifulSoup
from pandas.io.html import read_html
import pandas as pd

In [3]:
# pd.options.display.max_columns = None
# pd.options.display.max_rows = None

In [4]:
df = pd.DataFrame()
df.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

df = pd.read_csv("blankFile.csv", names= columnNames , header = None)
df = df.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    df[section] = ''
    df[sectionData] = ''

df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [5]:
# searchTerm1 = ['BIM', 'Building Information Modelling']
# searchTerm2 = ['Green Building', 'Green Buildings']

# searchTerm1 = ['a']
# searchTerm2 = ['the']

tc = 0
noResultsList = []

searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

counter = 0
programSearchTerms1 = []
programSearchTerms2 = []

for term2 in searchTerm2:
    text2 = ""
    terms2 = term2.split(' ')
    for t2 in terms2:
        text2 = text2 + t2 + "+"

    fileIndex = text2.rfind("+")
    te = text2[:fileIndex]
    programSearchTerms2.append(te)

print(programSearchTerms2)

for term1 in searchTerm1:
    text1 = ""
    terms1 = term1.split(' ')
    for t2 in terms1:
        text1 = text1 + t2 + "+"

    fileIndex = text1.rfind("+")
    te = text1[:fileIndex]
    programSearchTerms1.append(te)

print(programSearchTerms1)

# https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22
# BIM%22+AND+%22Green+Building
# %22&Ppub=&startPage=&SeriesKey=taem20

baseUrl = 'https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22'
querySet = []
row = 0
for item1 in programSearchTerms1:
    for item2 in programSearchTerms2:
        
        query = baseUrl + item1 +"%22+AND+%22"+ item2+"%22&Ppub=&startPage=&SeriesKey=taem20"
        print(query)
        response = requests.get(query)
        soup = BeautifulSoup(response.text, 'html.parser')
        allParas = soup.find_all("p")
        dd = allParas[0].text
        miniData = dd.split(' ')
        NumberOfResultsStr = miniData[2]
        indexOfcomma = NumberOfResultsStr.find(',')

        if indexOfcomma != -1:
            NumberOfResultsStr = NumberOfResultsStr.replace(',','')

        number = int(NumberOfResultsStr)
        tc = tc + number
        
        if number == 0:
            noResultsList.append((item1 +","+item2))
        
        page = number // 10
        lastPage = number % 10
        if lastPage > 0:
            page = page + 1

        pageSize = page
#         print(pageSize)

        cnt = 0
        allLinks = []
        
#         for testing purpose only
#         pageSize = 2

#         for currentPage in range(0,2):
        for currentPage in range(0,pageSize):
            print("In page: ",currentPage)
            
            if currentPage != 0:
                newQuery = query + '&pageSize=10&subjectTitle=&startPage=' + str(currentPage)
                cnt = cnt + 1
            else:
                newQuery = query
                        
            
            print("Searching for query: ",newQuery)
            response = requests.get(newQuery)
            soup = BeautifulSoup(response.text, 'html.parser')
            mydivs = soup.findAll("a", {'class':['ref nowrap']})
            
            for d in mydivs:
                url = 'https://www.tandfonline.com' + d.get('href')
                allLinks.append(url)

        for link in allLinks:
            df.loc[row,'searchTerm1'] = item1
            df.loc[row,'searchTerm2'] = item2
            df.loc[row,'url'] = link
            html_content = requests.get(link).text
            soup = BeautifulSoup(html_content, "lxml")
            
            for i in soup.find('span',{'class':['NLM_article-title' ,'hlFld-title']}):
#                 print("Title: ",i)
                df.loc[row,'Title'] = i
            
            temp = soup.findAll('h2',{'class':['section-heading-2']} )
            print("Number of sections: ",len(temp))            
            
            count = 0
            for i in range(len(temp)):

                if temp[i].text == temp[i].find_next('p').text:
        
                    extract = "section" + str(count)
                    extractData = temp[i].find_next('p').find_next('p').text
                    df.loc[row,extract] = temp[i].text
                    sectionData = "sectionData" + str(count)                    
                    df.loc[row,sectionData] = extractData
                    count = count + 1
                else:
                    extract = "section" + str(count)
                    extractData = temp[i].find_next('p').text
                    df.loc[row,extract] = temp[i].text
                    sectionData = "sectionData" + str(count)
                    df.loc[row,sectionData] = extractData
                    count = count + 1
            
            
            row = row + 1
            print("row: ",row)


['Green+Building', 'Green+Buildings', 'Sustainable+building', 'Sustainable+buildings', 'Environmentally+friendly+design', 'Sustainability', 'Sustainable+development', 'Energy', 'Water', 'Waste', 'Thermal+comfort', 'Electricity', 'Solar', 'Wind+power', 'Green+infrastructure', 'Life+cycle+assessment']
['BIM', 'Building+Information+Modelling', 'Building+information+model', '6D+BIM', 'Digital+construction', 'Smart+construction', 'Digital+engineering']
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22BIM%22+AND+%22Green+Building%22&Ppub=&startPage=&SeriesKey=taem20
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22BIM%22+AND+%22Green+Building%22&Ppub=&startPage=&SeriesKey=taem20
Number of sections:  11
row:  1
Number of sections:  11
row:  2
Number of sections:  10
row:  3
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22BIM%22+AND+%22Green+Buildings%22&Ppub=&startPage=&SeriesKey=taem20
In page:  0


https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Green+Buildings%22&Ppub=&startPage=&SeriesKey=taem20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Sustainable+building%22&Ppub=&startPage=&SeriesKey=taem20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Sustainable+buildings%22&Ppub=&startPage=&SeriesKey=taem20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Environmentally+friendly+design%22&Ppub=&startPage=&SeriesKey=taem20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Sustainability%22&Ppub=&startPage=&SeriesKey=taem20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Sustainable+development%22&Ppub=&startPage=&SeriesKey=taem20
https://www.

https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Smart+construction%22+AND+%22Sustainable+development%22&Ppub=&startPage=&SeriesKey=taem20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Smart+construction%22+AND+%22Energy%22&Ppub=&startPage=&SeriesKey=taem20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Smart+construction%22+AND+%22Water%22&Ppub=&startPage=&SeriesKey=taem20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Smart+construction%22+AND+%22Waste%22&Ppub=&startPage=&SeriesKey=taem20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Smart+construction%22+AND+%22Thermal+comfort%22&Ppub=&startPage=&SeriesKey=taem20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Smart+construction%22+AND+%22Electricity%22&Ppub=&startPage=&SeriesKey=taem20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Smart+construction%22+AND+%22Solar%22&Ppub=&startP

In [6]:
noResultsList

['BIM,Sustainable+buildings',
 'BIM,Environmentally+friendly+design',
 'BIM,Sustainable+development',
 'BIM,Water',
 'BIM,Waste',
 'BIM,Thermal+comfort',
 'BIM,Electricity',
 'BIM,Solar',
 'BIM,Wind+power',
 'BIM,Green+infrastructure',
 'BIM,Life+cycle+assessment',
 'Building+Information+Modelling,Sustainable+buildings',
 'Building+Information+Modelling,Environmentally+friendly+design',
 'Building+Information+Modelling,Sustainable+development',
 'Building+Information+Modelling,Water',
 'Building+Information+Modelling,Waste',
 'Building+Information+Modelling,Thermal+comfort',
 'Building+Information+Modelling,Electricity',
 'Building+Information+Modelling,Solar',
 'Building+Information+Modelling,Wind+power',
 'Building+Information+Modelling,Green+infrastructure',
 'Building+information+model,Green+Building',
 'Building+information+model,Green+Buildings',
 'Building+information+model,Sustainable+building',
 'Building+information+model,Sustainable+buildings',
 'Building+information+model,E

In [7]:
df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19
0,BIM,Green+Building,https://www.tandfonline.com/doi/full/10.1080/1...,Building information modelling and green build...,ABSTRACT,The advent of building information modelling (...,Introduction,Gauging the performance of a built environment...,Method,The objectives of this review needed a two-pha...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BIM,Green+Building,https://www.tandfonline.com/doi/full/10.1080/1...,Digitisation of existing buildings to support ...,ABSTRACT,Buildings’ functional and physical characteris...,Introduction,The construction industry is the major contrib...,Laser scanning in the built environment,Datasets from the laser scanner record real-wo...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BIM,Green+Building,https://www.tandfonline.com/doi/full/10.1080/1...,Green-building information modelling (Green-BI...,ABSTRACT,There is a research gap in the use of digital ...,Highlights,,1. Introduction,"In recent years, there has been an increase in...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BIM,Green+Buildings,https://www.tandfonline.com/doi/full/10.1080/1...,Building information modelling and green build...,ABSTRACT,The advent of building information modelling (...,Introduction,Gauging the performance of a built environment...,Method,The objectives of this review needed a two-pha...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BIM,Green+Buildings,https://www.tandfonline.com/doi/full/10.1080/1...,Building envelope trade-off method integrated ...,ABSTRACTFormulae display:?Mathematical formula...,ABSTRACT,Introduction,The United Nation's Agenda 2030 for sustainabl...,Literature review,This section discusses the uniqueness of ECBC ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,BIM,Sustainable+building,https://www.tandfonline.com/doi/full/10.1080/1...,Towards data-driven sustainable design: decisi...,ABSTRACT,Sustainable building design requires an interp...,Introduction,Sustainable building design requires an optima...,Data exchange and analysis in collaborative su...,"Sustainability is a multi-dimensional matter, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,BIM,Sustainability,https://www.tandfonline.com/doi/full/10.1080/1...,Building information modelling and green build...,ABSTRACT,The advent of building information modelling (...,Introduction,Gauging the performance of a built environment...,Method,The objectives of this review needed a two-pha...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,BIM,Sustainability,https://www.tandfonline.com/doi/full/10.1080/1...,Redefining the territory and competency of arc...,ABSTRACT,Within the digital era of building information...,Introduction,"In the new world economy, individual competenc...",Research method,This study adapted the PRISMA-P (Preferred Rep...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,BIM,Sustainability,https://www.tandfonline.com/doi/full/10.1080/1...,Digitisation of existing buildings to support ...,ABSTRACT,Buildings’ functional and physical characteris...,Introduction,The construction industry is the major contrib...,Laser scanning in the built environment,Datasets from the laser scanner record real-wo...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,BIM,Sustainability,https://www.tandfonline.com/doi/full/10.1080/1...,Green-building information modelling (Green-BI...,ABSTRACT,There is a research gap in the use of digital ...,Highlights,,1. Introduction,"In recent years, there has been an increase in...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.to_csv('kgRawData/KG-AEDM.csv')

In [5]:
import pandas as pd
df = pd.read_csv('kgRawData/KG-AEDM.csv')

In [6]:
df = df.where(pd.notnull(df), None)
df

,Unnamed: 0,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19
0,0,BIM,Green+Building,https://www.tandfonline.com/doi/full/10.1080/1...,Building information modelling and green build...,ABSTRACT,The advent of building information modelling (...,Introduction,Gauging the performance of a built environment...,Method,...,None,None,None,None,None,None,None,None,None,None
1,1,BIM,Green+Building,https://www.tandfonline.com/doi/full/10.1080/1...,Digitisation of existing buildings to support ...,ABSTRACT,Buildings’ functional and physical characteris...,Introduction,The construction industry is the major contrib...,Laser scanning in the built environment,...,None,None,None,None,None,None,None,None,None,None
2,2,BIM,Green+Building,https://www.tandfonline.com/doi/full/10.1080/1...,Green-building information modelling (Green-BI...,ABSTRACT,There is a research gap in the use of digital ...,Highlights,,1. Introduction,...,None,None,None,None,None,None,None,None,None,None
3,3,BIM,Green+Buildings,https://www.tandfonline.com/doi/full/10.1080/1...,Building information modelling and green build...,ABSTRACT,The advent of building information modelling (...,Introduction,Gauging the performance of a built environment...,Method,...,None,None,None,None,None,None,None,None,None,None
4,4,BIM,Green+Buildings,https://www.tandfonline.com/doi/full/10.1080/1...,Building envelope trade-off method integrated ...,ABSTRACTFormulae display:?Mathematical formula...,ABSTRACT,Introduction,The United Nation's Agenda 2030 for sustainabl...,Literature review,...,None,None,None,None,None,None,None,None,None,None
5,5,BIM,Sustainable+building,https://www.tandfonline.com/doi/full/10.1080/1...,Towards data-driven sustainable design: decisi...,ABSTRACT,Sustainable building design requires an interp...,Introduction,Sustainable building design requires an optima...,Data exchange and analysis in collaborative su...,...,None,None,None,None,None,None,None,None,None,None
6,6,BIM,Sustainability,https://www.tandfonline.com/doi/full/10.1080/1...,Building information modelling and green build...,ABSTRACT,The advent of building information modelling (...,Introduction,Gauging the performance of a built environment...,Method,...,None,None,None,None,None,None,None,None,None,None
7,7,BIM,Sustainability,https://www.tandfonline.com/doi/full/10.1080/1...,Redefining the territory and competency of arc...,ABSTRACT,Within the digital era of building information...,Introduction,"In the new world economy, individual competenc...",Research method,...,None,None,None,None,None,None,None,None,None,None
8,8,BIM,Sustainability,https://www.tandfonline.com/doi/full/10.1080/1...,Digitisation of existing buildings to support ...,ABSTRACT,Buildings’ functional and physical characteris...,Introduction,The construction industry is the major contrib...,Laser scanning in the built environment,...,None,None,None,None,None,None,None,None,None,None
9,9,BIM,Sustainability,https://www.tandfonline.com/doi/full/10.1080/1...,Green-building information modelling (Green-BI...,ABSTRACT,There is a research gap in the use of digital ...,Highlights,,1. Introduction,...,None,None,None,None,None,None,None,None,None,None


In [10]:
# df.to_csv('KG-AEDM-7476.csv')

In [8]:
import re
absRows = []
rowsFound = []
for i in range(len(df['url'])):
    text = df['url'][i]
    pattern = '/abs/'
    if re.search(pattern, text):
        absRows.append(i)
    else:
        rowsFound.append(i)
        

In [11]:
rowsFoundDf = pd.DataFrame()
rowsFoundDf.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

rowsFoundDf = pd.read_csv("blankFile.csv", names= columnNames , header = None)
rowsFoundDf = rowsFoundDf.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    rowsFoundDf[section] = ''
    rowsFoundDf[sectionData] = ''

    
absRowsDf = pd.DataFrame()
absRowsDf.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

absRowsDf = pd.read_csv("blankFile.csv", names= columnNames , header = None)
absRowsDf = absRowsDf.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    absRowsDf[section] = ''
    absRowsDf[sectionData] = ''
    
    
for entry in range(len(rowsFound)):
    rowsFoundDf.loc[entry] = df.loc[rowsFound[entry]]

for entry in range(len(absRows)):
    absRowsDf.loc[entry] = df.loc[absRows[entry]]

rowsFoundDf.to_csv('kgRawData/tandf/kg-aedm-full.csv')
absRowsDf.to_csv('kgRawData/tandf/kg-aedm-abs.csv')

print(len(absRowsDf))
print(len(rowsFoundDf))

2
24


In [13]:
absRowsDf

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19
0,BIM,Energy,https://www.tandfonline.com/doi/abs/10.3763/ae...,Exchange of Relevant Information in BIM Object...,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Building+information+model,Energy,https://www.tandfonline.com/doi/abs/10.3763/ae...,Exchange of Relevant Information in BIM Object...,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
